In [0]:
import pandas as pd
import numpy as np
import csv
from google.colab import files
import io

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample


import matplotlib.pyplot as plt

#from keras.models import Sequential
#from keras.layers import Dense


In [22]:
uploaded = files.upload()

Saving SubmissionFile.csv to SubmissionFile.csv
Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [18]:
train_values_df = pd.read_csv(io.BytesIO(uploaded['train.csv']))
submission_values_df = pd.read_csv(io.BytesIO(uploaded['test.csv']))


#Balancing Classes
df_majority = train_values_df[train_values_df.Survived==0]
df_minority = train_values_df[train_values_df.Survived==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=549,    # to match majority class
                                 random_state=123) # reproducible results

train_values_df = pd.concat([df_majority, df_minority_upsampled])

#Dividing Data

data_X = train_values_df.iloc [:,[2,4,5,6,7]]
data_Y = train_values_df.iloc [:,1].to_numpy()

submission_X = submission_values_df.iloc[:,[1,3,4,5,6]]


#Checking for class imbalance
counter_0,counter_1 = 0,0
for vals in data_Y:
  if vals == 0:
    counter_0+=1
  else:
    counter_1+=1
  
print(counter_0,counter_1)


scaler = MinMaxScaler()

#Label Encoding
label_encoder = LabelEncoder()
data_X = data_X.apply(label_encoder.fit_transform)
scaler.fit(data_X[['Age']])
age_vector = scaler.transform(data_X[['Age']])
data_X['Age'] = age_vector


submission_X = submission_X.apply(label_encoder.fit_transform)
scaler.fit(submission_X[['Age']])
sub_age_vector = scaler.transform(submission_X[['Age']])
submission_X['Age'] = sub_age_vector


#Updated array to fit One Hot Encoder
label_encoder_fit_array = pd.concat([data_X,submission_X])
label_encoder_fit_array = label_encoder_fit_array.to_numpy()

#Converting individual to numpy arrays after combining completed above
data_X = data_X.to_numpy()
submission_X = submission_X.to_numpy()


#Updated One Hot Encoding
my_encoder_X = OneHotEncoder(categorical_features = [0,1,3,4])
my_encoder_X.fit(label_encoder_fit_array)
data_X_transformed = my_encoder_X.transform(data_X).toarray()
submission_transformed = my_encoder_X.transform(submission_X).toarray()


#One Hot Encoding
#my_encoder_X = OneHotEncoder(categorical_features = [0,1,3,4])
#data_X_transformed = my_encoder_X.fit_transform(data_X).toarray()
#submission_transformed = my_encoder_X.fit_transform(submission_X).toarray()

my_encoder_Y = OneHotEncoder()
data_Y_transformed = my_encoder_Y.fit_transform(data_Y.reshape(-1,1)).toarray()


#Train_Test_Split
X_train, X_test, Y_train, Y_test = train_test_split(data_X_transformed, data_Y, test_size=0.30, random_state=40)

#######ADDRESS THE ISSUE WITH DIFFERENT SIZES OF TRAIN SET AND SUBMISSION SET BECAUSE OF A MISSING DATA POINT WITH SPECIFIC CLASS VALUE IN ONE OF THE FEATURES ---> This causes difference in shape! #######
print(data_X_transformed.shape)
#print(data_X_transformed)
print(submission_transformed.shape)
#print(submission_transformed)

549 549
(1098, 21)
(418, 21)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data w

In [19]:
#Logistic Regression
logistic_regression = LogisticRegression(solver='lbfgs', penalty="none", fit_intercept=False)

logistic_regression.fit(X_train,Y_train)
predictions = logistic_regression.predict(X_test)
score = accuracy_score(predictions,Y_test) * 100
print(score)
print(classification_report(predictions,Y_test))

80.0
              precision    recall  f1-score   support

           0       0.86      0.77      0.81       184
           1       0.74      0.84      0.79       146

    accuracy                           0.80       330
   macro avg       0.80      0.80      0.80       330
weighted avg       0.81      0.80      0.80       330



In [0]:
#Decision Tree
decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini',max_depth=5, class_weight = 'balanced')

decision_tree.fit(X_train,Y_train)
predictions = decision_tree.predict(X_test)
score = accuracy_score(predictions,Y_test) * 100
print(score)
print(classification_report(predictions,Y_test))

79.39393939393939
              precision    recall  f1-score   support

           0       0.86      0.76      0.81       186
           1       0.73      0.83      0.78       144

    accuracy                           0.79       330
   macro avg       0.79      0.80      0.79       330
weighted avg       0.80      0.79      0.79       330



In [26]:
#Submission File Preparation

#Writing Logistic Regression Results on the Submission CSV file. 
logistic_regression = LogisticRegression(solver='lbfgs', penalty="none", fit_intercept=False)
logistic_regression.fit(X_train,Y_train)
predictions = logistic_regression.predict(submission_transformed)
#print(prob_pred)

output_list = []
with open('SubmissionFile.csv') as csv_file:
  reader = csv.reader(csv_file)
  output_list = list(reader)
csv_file.close()


pred_list = predictions.tolist()
print(pred_list)

output_list[0].append('Survived')
counter = 1
for items in pred_list:
  output_list[counter].append(items)
  counter+=1
  
print(output_list)


#writing output list to csv_file
with open("FinalFile.csv",'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerows(output_list)
csv_file.close()

#files.download("FinalFile.csv")

[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 